# Prediction with LSTM model
<p style='text-align: right;'>with selectd1.csv</p>

* history
  * 2025/05/14 PM06:56 : 모델에 대한 기능을 공통화 시킴

In [7]:
def restart_kernel():
    # Restart the kernet after libraries are loaded.
    import IPython
    from datetime import datetime
    print(f'restart kernel... {datetime.now()}')
    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

In [ ]:
!pip install scikit-learn
!pip install tensorflow
from datetime import datetime
print(f'restart kernel... {datetime.now()}')

In [8]:
restart_kernel()

restart kernel... 2025-05-15 17:06:07.360561


In [2]:
# load dependacies
import os
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from matplotlib import pyplot
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Input

from globalvar import *
from data import DataLoader
from model1 import PredictLSTM1
from model2 import PredictLSTM2
from model3 import PredictLSTM3
from model4 import PredictLSTM4

import argparse
import matplotlib.pyplot as plt

from util_pred import print_data
from util_pred import flat_data_with_sum, flat_data, get_frequency
from util_pred import save_model, import_mode
from util_pred import get_random_in_list
from util_pred import print_data_with_sort
from util_pred import get_sorted_n_values
from util_pred import dict_key_count
from util_pred import print_list
from util_pred import print_dict_list
from util_pred import print_title
from activation import ActivationOutput, RecurrentActivation
from datetime import datetime

from operate import create_model, create_model_v1, get_predicted

print(f"imported library. ({datetime.now()})")

2025-05-16 15:32:48.755510: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


imported library. (2025-05-16 15:32:50.463520)


In [101]:
##
## 모델링 환경 설정
##
window=1 # default = 1 , help = "time stamps"
data_dir='./selectd2.csv'
    
mode='predict' # help = "back-test or predict")
mode2='sampling' # help = "greed or sampling")
verb='verbose' # help = "verbose or not_verb")
    
trial=20 # help = "how much trials to generate")
training_length=1 # default = 0.9)
epoch=20 # default = 3
batch=1 # default = 1
model_type='lstm4' # help = "lstm1 or lstm2")
hid_dim = 50
from_pos = 0
last = [[3, 6, 7, 11, 12, 17],
        [3, 13, 28, 34, 38, 42],
        [5, 12, 24, 29, 32, 42]]
print(f"finished to set environemnt. ({datetime.now()})")
MAX_MODEL_CNT = 6

finished to set environemnt. (2025-05-16 16:53:50.454078)


In [7]:
# data prepare and set base model
dataset = DataLoader(data_dir=data_dir,
                     training_length=training_length,
                     window_prev=window,
                     mode=mode,
                     from_pos=from_pos
                    )

dataset_dicts = {
    1: DataLoader(data_dir=data_dir,
                  training_length=training_length,
                  window_prev=window,
                  mode=mode,
                  from_pos=0
                  ),
    2: DataLoader(data_dir=data_dir,
                  training_length=training_length,
                  window_prev=window,
                  mode=mode,
                  from_pos=300
                  )
    }
print(f'completed to load data. {datetime.now()}')

DataLoader.preproc_csv origin = 570, [[ 601    2   16 ...   31   34   35]
 [ 602   13   14 ...   27   30   38]
 [ 603    2   19 ...   26   27   43]
 ...
 [1168    9   21 ...   30   33   37]
 [1169    5   12 ...   26   39   42]
 [1170    3   13 ...   34   38   42]]
rDataLoader.preproc_csv aw_np_proc = 570, [[16 19 31 34 35]
 [14 22 27 30 38]
 [19 25 26 27 43]
 ...
 [21 24 30 33 37]
 [12 24 26 39 42]
 [13 28 34 38 42]]
DataLoader.preproc_csv origin = 570, [[ 601    2   16 ...   31   34   35]
 [ 602   13   14 ...   27   30   38]
 [ 603    2   19 ...   26   27   43]
 ...
 [1168    9   21 ...   30   33   37]
 [1169    5   12 ...   26   39   42]
 [1170    3   13 ...   34   38   42]]
rDataLoader.preproc_csv aw_np_proc = 570, [[16 19 31 34 35]
 [14 22 27 30 38]
 [19 25 26 27 43]
 ...
 [21 24 30 33 37]
 [12 24 26 39 42]
 [13 28 34 38 42]]
DataLoader.preproc_csv origin = 570, [[ 601    2   16 ...   31   34   35]
 [ 602   13   14 ...   27   30   38]
 [ 603    2   19 ...   26   27   43]
 ...
 [116

In [105]:
###########
## layers
## LSTM Neural 계층 선언
###########
"""
Activation (Output):
    linear: No activation, output is directly passed through.
    relu: Rectified Linear Unit, max(x, 0).
    sigmoid: Sigmoid function, output between 0 and 1.
    tanh: Hyperbolic tangent, output between -1 and 1.
    softmax: Normalizes output to a probability distribution.
    elu: Exponential Linear Unit.
    selu: Scaled Exponential Linear Unit.

Recurrent Activation:
    sigmoid: Commonly used for gates in LSTM.
    hard_sigmoid: A faster, less computationally expensive version of sigmoid.
    tanh: Can be used, but sigmoid is more typical for gates.
"""
lstm_layers = {
    4: [LSTM(hid_dim,
             activation=ActivationOutput.selu.name,
             return_sequences=True),
        LSTM(hid_dim,
             return_sequences=True,
             activation=ActivationOutput.selu.name),
        LSTM(hid_dim,
             return_sequences=True,
             activation=ActivationOutput.sigmoid.name),
        LSTM(hid_dim,
             return_sequences=False,
             activation=ActivationOutput.elu.name,
             recurrent_activation="hard_sigmoid")],
    }

simple_lstm_layer = [LSTM(hid_dim,
                     activation=ActivationOutput.elu.name)]

dense_layers = {0: [Dense(45, activation='softmax'),
                    Dense(45, activation='sigmoid')],
                7: [Dense(45, activation='softmax')]
               }
simple_dense_layer = [Dense(45, activation='softmax')]

layers = [
    lstm_layers[4] + dense_layers[7], # 4
    lstm_layers[4] + dense_layers[0], # 5
    ]

datasets = [dataset_dicts[1], # 4
            dataset_dicts[2], # 5
           ]

epochs = [20, 10]

models = [None for i in range(MAX_MODEL_CNT)]
#matched_cnts = [0 for i in range(MAX_MODEL_CNT)]
#selected_fives = [None for i in range(MAX_MODEL_CNT)]
#matched_list = [None for i in range(MAX_MODEL_CNT)]
#predicted_all = [None for i in range(MAX_MODEL_CNT)]

matched_cnts = {}
selected_fives = {}
matched_list = {}
predicted_all = {}

print(f'completed to set evn for all models. {datetime.now()}')

completed to set evn for all models. 2025-05-16 16:55:52.725467


In [106]:
# all : create model and training
print(f'start to train all models. {datetime.now()}')
for i in range(MAX_MODEL_CNT):
    print(f'{i+1}\'s training. status=start {datetime.now()}')
    model = create_model(id=i,
                         model_type="lstm4",
                         layers=layers[i],
                         dataset=datasets[i],
                         hid_dim=hid_dim,
                         epoch=epochs[i],
                         verbose=False
                        )
    models[i] = (model, layers[i])
    print(f'{i+1}\'s training. status=end {datetime.now()}')
print(f'completed to train all models. {datetime.now()}')

start to train all models. 2025-05-16 16:56:08.143899
1's training. status=start 2025-05-16 16:56:08.144059
1's training. status=end 2025-05-16 16:56:40.976720
2's training. status=start 2025-05-16 16:56:40.976817
2's training. status=end 2025-05-16 16:57:42.146504
3's training. status=start 2025-05-16 16:57:42.146606
3's training. status=end 2025-05-16 16:57:56.956036
4's training. status=start 2025-05-16 16:57:56.956146
4's training. status=end 2025-05-16 16:58:22.574932
5's training. status=start 2025-05-16 16:58:22.575032
5's training. status=end 2025-05-16 16:58:31.744049
6's training. status=start 2025-05-16 16:58:31.744170
6's training. status=end 2025-05-16 16:58:46.533942
completed to train all models. 2025-05-16 16:58:46.534216


In [107]:
print(f'check to create model. {datetime.now()}')

check to create model. 2025-05-16 16:59:31.699518


In [ ]:
# all : predict #1
greed_prediction = [None for i in range(MAX_MODEL_CNT)]
random_pred = [None for i in range(MAX_MODEL_CNT)]
for i in range(MAX_MODEL_CNT):
    greed_prediction[i] = models[i][0].predict_numbers("greed", trial=1)
    random_pred[i] = models[i][0].predict_randomely(trial=1)
    print(f'{i+1}\'s greed_prediction = {greed_prediction[i]}')
    print(f'{i+1}\'s random_pred = {random_pred[i]}')
print(f'comleted to predict model. {datetime.now()}')

In [136]:
def predict_all_model(model_cnt=MAX_MODEL_CNT):
    matched_cnts = [0 for i in range(MAX_MODEL_CNT)]
    selected_fives = [None for i in range(MAX_MODEL_CNT)]
    matched_list = [None for i in range(MAX_MODEL_CNT)]
    predicted_all = [None for i in range(MAX_MODEL_CNT)]
    for i in range(MAX_MODEL_CNT):
        title = f"history #{i+1}"
        if models[i] is not None:
            models[i][0].verb = "None"
            matched_cnts[i], \
            selected_fives[i], \
            matched_list[i], \
            predicted_all[i] = get_predicted(
                title=title,
                model=models[i][0],
                mode=mode2,
                use_pre=False,
                last=last,
                verbose=False,
                trial=5
                )
    return matched_cnts, selected_fives, matched_list, predicted_all


def repeat_predict_all_model(trial=1, model_cnt=MAX_MODEL_CNT):
    """ repeat_predict_all_model """
    matched_cnts = {}
    selected_fives = {}
    matched_list = {}
    predicted_all = {}
    for t in range(trial):
        matched_cnts[t], \
        selected_fives[t], \
        matched_list[t], \
        predicted_all[t] = predict_all_model(model_cnt)
    return matched_cnts, selected_fives, matched_list, predicted_all


In [ ]:
matched_cnts, selected_fives, matched_list, predicted_all = repeat_predict_all_model(trial=20)

In [ ]:
print(matched_cnts)
print(matched_list)

In [10]:
import json
import os
from datetime import datetime

def listinlist_2_strinlist(list_in_list):
    """ listinlist_2_strinlist """
    str_in_list = []
    for l in list_in_list:
        str_in_list.append(",".join(l))
    return str_in_list      


def change_matched_info(matched_count, matched_list):
    counts = {}
    cnt = 1
    for c in matched_count:
        counts[cnt] = ",".join([ str(i) for i in c])
        cnt += 1
    cnt = 1
    datas = {}
    for d in matched_list:
        datas[cnt] = d
        cnt += 1
    return counts, datas

def write_json(trial, matched_count, matched_list, append=False):
    suffix = datetime.now().strftime('%y%m%d_%H')
    file_name = f"matched_{suffix}.json"
    matched_dict = {}
    if append:
        if os.path.isfile(file_name):
            with open(file_name, "r") as file:
                matched_dict = json.load(file)
                file.close()
    counts, datas = change_matched_info(matched_count, matched_list)
    if len(counts) > 0 and len(datas) > 0:
        matched_dict[trial] = {}
        matched_dict[trial]["counts"] =  counts
        matched_dict[trial]["datas"] = datas
        print(f'matched_dict = {matched_dict}')
        with open(file_name, "w") as file:
            json.dump(matched_dict, file, indent=4)

In [ ]:
# all with trial : print predicted
write_json(trial = 3, matched_count=matched_cnts, matched_list=matched_list, append=True)
print_dict_list(title="matched_cnt", datas=matched_cnts)
for i in range(MAX_MODEL_CNT):
    print_list(title=f"matched_list[{i+1}]", datas=matched_list[i])

4시 40분 경 데이터
| 회차 | 3이상 ||| 0이 없음 |
| -- | -- | -- | -- | -- |
|  1 | N ||| N |
|  2 | 4, 5 ||| 3 |
|  3 | N ||| 3 |
|  4 | 2 ||| 5 |
|  5 | 4 ||| 1, 6 |
|  6 | 5 ||| N |
|  7 | N ||| 4 |
|  8 | N ||| N |
|  9 | 2 ||| N |
| 10 | 6 ||| 2 |
| 11 | N ||| 2 |
| 12 | **2** ||| **2**, 4 |
| 13 | 4 ||| N |
| 14 | 5 ||| N |
| 15 | 1*, 6 ||| 3 |
| 16 | 5* ||| N |
| 17 | N ||| 6 |
| 18 | N ||| N |
| 19 | 2 ||| 5 |
| 20 | N ||| N |
||||||
| 21 | N ||| N |
| 22 | N ||| 2 |
| 23 | 3 ||| N |
| 24 | N ||| N |
| 25 | 2 ||| N |
| 26 | N ||| N |
| 27 | N ||| N |
| 28 | 3 ||| 5 |
| 29 | N ||| 2 |
| 30 | N ||| N |
| 31 | **3** ||| **3** |
| 32 | N ||| N |
| 33 | N ||| 3, 5 |
| 34 | 6 ||| N |
| 35 | N ||| N |
| 36 | N ||| N |
| 37 | N ||| N |
| 38 | 1, 2 ||| N |
| 39 | N ||| 6 |
| 40 | **5** ||| **5** |
| 41 | 6 ||| N |

In [145]:
# all (SMALL): predict #2
matched_cnts_small = [0 for i in range(MAX_MODEL_CNT)]
selected_fives_small = [None for i in range(MAX_MODEL_CNT)]
matched_list_small = [None for i in range(MAX_MODEL_CNT)]
predicted_all_small = [None for i in range(MAX_MODEL_CNT)]

for i in range(MAX_MODEL_CNT):
    title = f"history #{i+1}"
    model_num = i
    if models[model_num] is None:
        print(f"model #{model_num+1} is None")
    else:
        models[model_num][0].verb = "None"
        matched_cnts_small[model_num], \
        selected_fives_small[model_num], \
        matched_list_small[model_num], \
        predicted_all_small[model_num] = get_predicted(
            title=title,
            model=models[model_num][0],
            mode=mode2,
            use_pre=False,
            last=last,
            verbose=False,
            trial=5
        )
print(f'predicted time : {datetime.now()}')

9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
predicted time : 2025-05-16 17:12:59.484490


| 회차 | 3이상 ||| 0이 없음 |
| -- | -- | -- | -- | -- |
|  1 | N ||| 3 |
|  2 | N ||| 2 |
|  3 | N ||| 5 |
|  4 | N ||| 4, 6 |
|  5 | 1 ||| 3 |
|  6 | 4, 6 ||| 3 |
|  7 | N ||| N |
|  8 | 3 ||| N |
|  9 | 4 ||| N |
| 10 | N ||| N |
| 11 | 3 ||| N |
| 12 | N ||| N |
| 13 | 4 ||| N |
| 14 | 1, 5, 6 ||| N |
| 15 | N ||| N |
| 16 | N ||| 1 |
| 17 | 4 ||| N |
| 18 | 4, 6 ||| 5 |
| 19 | 1, 5 ||| N |
| 20 | 1, 5, 5, 5 ||| 1 |

{14:6, 13:8, 12:4, _11:10_, 10:3, 9:1, 8:6, 7:5, 6:4, 5:9, 4:4, 3:7, 2:7, 1:4}
## 4시 40분경 데이터
- {2: 6, 5: 5, 6: 4, 4: 3, 3: 3, 1: 2}
- {6: 4, 5: 5, 4: 3, 3: 3, 2: 6, 1: 2}

## 5시 20분경 데이터
- {4: 5, 5: 5, 1: 4, 6: 3, 3: 2}
- {6: 3, 5: 5, 4: 5, 3: 2, 1: 4}

In [146]:
# all (SMALL): print predicted
print(f'print time : {datetime.now()}')
#write_json(trial = 3, matched_count=matched_cnts_small, matched_list=matched_list_small, append=True)
print_dict_list(title="matched_cnt", datas=matched_cnts_small)
for i in range(MAX_MODEL_CNT):
    print_list(title=f"matched_list[{i+1}]", datas=matched_list_small[i])

print time : 2025-05-16 17:13:01.893074
matched_cnt
	1's : {1: 3, 2: 1, 3: 1}
	2's : {0: 1, 1: 3, 2: 1}
	3's : {0: 3, 1: 2}
	4's : {0: 1, 1: 3, 2: 1}
	5's : {0: 1, 2: 1, 3: 2, 4: 1}
	6's : {0: 2, 1: 1, 2: 2}
matched_list[1]
	([6, 15, 27, 29, 36, 42], [6])
	([4, 16, 18, 39, 11, 6], [6, 11])
	([1, 2, 6, 13, 19, 25], [6])
	([1, 3, 29, 33, 36, 38], [3])
	([3, 6, 7, 9, 38, 41], [3, 6, 7])
matched_list[2]
	([3, 5, 21, 34, 40, 45], [3])
	([2, 4, 5, 16, 36, 37], [])
	([1, 43, 3, 13, 40, 4], [3])
	([6, 7, 23, 27, 34, 36], [6, 7])
	([11, 16, 24, 33, 43, 45], [11])
matched_list[3]
	([11, 14, 25, 28, 30, 43], [11])
	([5, 10, 16, 18, 26, 42], [])
	([10, 37, 8, 22, 40, 34], [])
	([4, 12, 22, 30, 38, 42], [12])
	([31, 4, 30, 16, 33, 23], [])
matched_list[4]
	([41, 21, 4, 11, 43, 31], [11])
	([5, 8, 17, 21, 23, 35], [17])
	([16, 18, 29, 38, 41, 33], [])
	([5, 7, 12, 32, 33, 38], [7, 12])
	([14, 17, 20, 24, 36, 45], [17])
matched_list[5]
	([6, 7, 11, 17, 23, 42], [6, 7, 11, 17])
	([18, 21, 22, 25, 26, 

In [ ]:
# all : save model
for model in models:
    if model is not None:
        model[0].save()

In [ ]:
# all : model info
#################
# model info
#################
for model in models:
    if model is not None:
        print(f'model:\n\t{model[0].model.summary()}')
        print(f'layer:\n\t{model[1]}')

In [55]:
# all : model test
mode = "back-test"
if mode == 'back-test':
    for model in models:
        if model is not None:
            greed_prediction_number_set = model[0].predict_numbers("greed", trial=1)
            random_pred_set = model[0].predict_randomely(trial=1)
            model[0].evaluate(greed_prediction_number_set)
            print("---------Random baseline-------------")
            model[0].evaluate(random_pred_set)

In [ ]:
for i in range(3):
    print(f'{i} : {get_random_in_list(prediction_number_set, 5)}')

In [ ]:
for i in range(5):
    print(f'{i} : {get_random_in_list(prediction_number_set, 5)}')

In [ ]:
# one : create model and training
id = 6
print(f'start to train. [id={id}]')
models[6] = create_model_v1(id=6, dataset=dataset, epoch=10, verbose=True)
print(f'model = {models[6]}')
print(f'end to train. [id={id}]')

In [ ]:
# predict #1
title = "history #1"
model_num = 0 # real 1
if models[model_num] is None:
    print(f"model #{model_num+1} is None")
else:
    models[model_num][0].verb = "None"
    matched_cnts[model_num], \
    selected_fives[model_num], \
    matched_list[model_num], \
    predicted_all[model_num] = get_predicted(
        title=title,
        model=models[model_num][0],
        mode=mode2,
        use_pre=False,
        last=last,
        verbose=True,
        trial=10
        )

In [ ]:
# predict #2
title = "history #2"
model_num = 1 # real 2
if models[model_num] is None:
    print(f"model #{model_num+1} is None")
else:
    models[model_num][0].verb = "None"
    matched_cnts[model_num], \
    selected_fives[model_num], \
    matched_list[model_num], \
    predicted_all[model_num] = get_predicted(
        title=title,
        model=models[model_num][0],
        mode=mode2,
        use_pre=False,
        last=last,
        verbose=True,
        trial=10
        )

In [ ]:
# predict #3
title = "history #3"
model_num = 2 # real 3
if models[model_num] is None:
    print(f"model #{model_num+1} is None")
else:
    models[model_num][0].verb = "None"
    matched_cnts[model_num], \
    selected_fives[model_num], \
    matched_list[model_num], \
    predicted_all[model_num] = get_predicted(
        title=title,
        model=models[model_num][0],
        mode=mode2,
        use_pre=False,
        last=last,
        verbose=True,
        trial=10
        )

In [ ]:
# predict #4
title = "history #4"
model_num = 3 # real 4
if models[model_num] is None:
    print(f"model #{model_num+1} is None")
else:
    models[model_num][0].verb = "None"
    matched_cnts[model_num], \
    selected_fives[model_num], \
    matched_list[model_num], \
    predicted_all[model_num] = get_predicted(
        title=title,
        model=models[model_num][0],
        mode=mode2,
        use_pre=False,
        last=last,
        verbose=True,
        trial=10
        )

In [ ]:
# predict #5
title = "history #5"
model_num = 4 # real 5
if models[model_num] is None:
    print(f"model #{model_num+1} is None")
else:
    models[model_num][0].verb = "None"
    matched_cnts[model_num], \
    selected_fives[model_num], \
    matched_list[model_num], \
    predicted_all[model_num] = get_predicted(
        title=title,
        model=models[model_num][0],
        mode=mode2,
        use_pre=False,
        last=last,
        verbose=True,
        trial=10
        )

In [ ]:
# predict #6
title = "history #6"
model_num = 5 # real 6
if models[model_num] is None:
    print(f"model #{model_num+1} is None")
else:
    models[model_num][0].verb = "None"
    matched_cnts[model_num], \
    selected_fives[model_num], \
    matched_list[model_num], \
    predicted_all[model_num] = get_predicted(
        title=title,
        model=models[model_num][0],
        mode=mode2,
        use_pre=False,
        last=last,
        verbose=True,
        trial=10
        )

In [22]:
# [3, 6, 7, 11, 12, 17]

In [29]:
new1 = flat_data(prediction_number_set22)
new_dict = {}
for n in new1:
    if n in new_dict:
        new_dict[n] += 1
    else:
        new_dict[n] = 1
# print(f'new_dict = {new_dict}')
top, all = get_sorted_n_values(new_dict)
print(f'all = {all}')
print(f'top = {top}')
# print(f'new_dict = {new_dict}')
bottom, all = get_sorted_n_values(new_dict, is_sort=False)
print(f'bottom = {bottom}')

NameError: name 'prediction_number_set22' is not defined

In [ ]:
for i in range(5):
    print(f'{i} : {get_random_in_list(prediction_number_set2, 5)}')

In [ ]:
print_data(title='greed_predicted',
           data_set=greed_prediction_number_set,
           add_val=0, need_sort=True)
print(f'greed_prediction_number_set\n\t\t{greed_prediction_number_set}')

In [ ]:
# when 
"""
trial=20 # help = "how much trials to generate")
training_length=1 # default = 0.9)
epoch=100 # default = 3
batch=3 # default = 1
model='lstm2' # help = "lstm or lstm2")
hid_dim = 128
"""
for i in range(6):
    print(f'{i} : {get_random_in_list(prediction_number_set, 5)}')

In [ ]:
print(f'model={model}')

In [ ]:
random_pred_set = model2.predict_randomely(trial=1)
print(f'prediction_number_set = {random_pred_set}')

In [ ]:
# 1170 : 3·13·28·34·38·42

last = [
    [6, 8, 37, 40, 41, 44],
    [8, 10, 20, 25, 33, 37],
    [4, 18, 27, 32, 40, 43],
    [2, 8, 10, 31, 33, 35],
    [15, 20, 28, 38, 41, 45],
    [12, 26, 13, 5, 31, 38]
]

In [ ]:
#1171 : [3, 6, 7, 11, 12, 17]